In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [3]:
# 定义数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 加载训练集和测试集
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 创建数据加载器
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [4]:
# 定义 CNN 模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
# 初始化模型
model = CNN()

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
    print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {running_loss/len(train_loader):.4f}')

Epoch [1/5], Step [100/938], Loss: 0.3334
Epoch [1/5], Step [200/938], Loss: 0.2269
Epoch [1/5], Step [300/938], Loss: 0.1368
Epoch [1/5], Step [400/938], Loss: 0.1760
Epoch [1/5], Step [500/938], Loss: 0.1157
Epoch [1/5], Step [600/938], Loss: 0.0925
Epoch [1/5], Step [700/938], Loss: 0.0858
Epoch [1/5], Step [800/938], Loss: 0.1243
Epoch [1/5], Step [900/938], Loss: 0.0099
Epoch [1/5], Average Loss: 0.1592
Epoch [2/5], Step [100/938], Loss: 0.0794
Epoch [2/5], Step [200/938], Loss: 0.0074
Epoch [2/5], Step [300/938], Loss: 0.0201
Epoch [2/5], Step [400/938], Loss: 0.0153
Epoch [2/5], Step [500/938], Loss: 0.0263
Epoch [2/5], Step [600/938], Loss: 0.1508
Epoch [2/5], Step [700/938], Loss: 0.0275
Epoch [2/5], Step [800/938], Loss: 0.0318
Epoch [2/5], Step [900/938], Loss: 0.0023
Epoch [2/5], Average Loss: 0.0451
Epoch [3/5], Step [100/938], Loss: 0.0069
Epoch [3/5], Step [200/938], Loss: 0.0068
Epoch [3/5], Step [300/938], Loss: 0.0646
Epoch [3/5], Step [400/938], Loss: 0.0825
Epoch [3

In [6]:
# 测试模型
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')

Test Accuracy: 99.20%
